In this notebook we want to develop some functions and intermediate datasets that we know will be correct. We will use these functions and datasets to check that our speed testing is correct.

In [1]:
# import libraries and data

import cbitstruct as bitstruct
import numpy as np
import pandas as pd

import pixie16

bin_file = "pixie16_binary_data.bin"

In [2]:
# load in the events correctly parsed

correct_events = pixie16.read.read_list_mode_data([bin_file])
correct_events.tail()

,channel,crate,slot,timestamp,CFD_fraction,energy,trace,CFD_error,pileup,trace_flag,...,Esum_gap,baseline,QDCSum0,QDCSum1,QDCSum2,QDCSum3,QDCSum4,QDCSum5,QDCSum6,QDCSum7
24593,10,0,2,1180571693780,0.649658,16785,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
24594,10,0,2,1180571922190,-1.649170,4060,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
24595,10,0,2,1180571988040,0.000000,1832,[],True,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
24596,9,0,2,1180572262810,3.884521,26102,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
24597,10,0,2,1180572322710,0.000000,1757,[],True,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1


In [3]:
# now create the intermediate dataset required to make this complete one

HEADER_FORMATS = {
    0: (
        "b1u14u5u4u4u4",
        ("Finish Code", "Event Length", "Header Length", "CrateID", "SlotID", "Chan#"),
    ),
    1: ("u32", ("EVTTIME_LO",)),
    2: ("u3u13u16", ("CFD trigger source bits", "CFD Fractional Time", "EVTTIME_HI")),
    3: ("b1u15u16", ("Trace Out-of-Range Flag", "Trace Length", "Event Energy")),
}


bit_fmt = ""
nbits = 0
field_names = ()

for i in HEADER_FORMATS.keys():
    bit_fmt += HEADER_FORMATS[i][0]
    nbits += bitstruct.calcsize(bit_fmt)
    field_names += HEADER_FORMATS[i][1]
    
field_idxs = {name: i for i, name in enumerate(field_names)}

HEADER_LENGTH = bitstruct.calcsize(bit_fmt) // 8


def read_list_mode_data_to_bit_fields(file):
    events = []
    with open(file, "rb") as f:
        while raw_header_bits := f.read(HEADER_LENGTH):
            try:
                header_bits = bitstruct.byteswap(HEADER_LENGTH // 4 * '4', raw_header_bits)
                events.append(bitstruct.unpack(bit_fmt, header_bits))
            except TypeError:
                break
    return pd.DataFrame.from_records(events, columns=field_names)


# table for conversion from pixie manual field names to field names used by pixie16 python module


column_names_tbl = {
    'Event Energy': 'energy',
    'Trace Out-of-Range Flag': 'trace_flag',
    'Chan#': 'channel',
    'CrateID': 'crate',
    'SlotID': 'slot'
}


def read_list_mode_data_to_completed_fields(file):
    df = read_list_mode_data_to_bit_fields(file)
    df['timestamp'] = (df['EVTTIME_LO'] + df['EVTTIME_HI'] * 2 ** 32) * 10
    df['CFD_error'] = df['CFD trigger source bits'] == 7
    df['CFD_fraction'] = np.where(df['CFD_error'], 0, ((df['CFD trigger source bits'] - 1) + df['CFD Fractional Time'] / 8192) * 2)
    return df.rename(columns=column_names_tbl)

In [4]:
read_events = read_list_mode_data_to_completed_fields(bin_file)
read_events.tail()

,Finish Code,Event Length,Header Length,crate,slot,channel,EVTTIME_LO,CFD trigger source bits,CFD Fractional Time,EVTTIME_HI,trace_flag,Trace Length,energy,timestamp,CFD_error,CFD_fraction
24593,False,4,4,0,2,10,2093052386,1,2661,27,False,0,16785,1180571693780,False,0.649658
24594,False,4,4,0,2,10,2093075227,0,1437,27,False,0,4060,1180571922190,False,-1.649170
24595,False,4,4,0,2,10,2093081812,7,0,27,False,0,1832,1180571988040,True,0.000000
24596,False,4,4,0,2,9,2093109289,2,7719,27,False,0,26102,1180572262810,False,3.884521
24597,False,4,4,0,2,10,2093115279,7,0,27,False,0,1757,1180572322710,True,0.000000


In [5]:
for idx in read_events.columns & correct_events.columns:
    print(idx, ': ', all(read_events[idx] == correct_events[idx]))

crate :  True
slot :  True
channel :  True
trace_flag :  True
energy :  True
timestamp :  True
CFD_error :  True
CFD_fraction :  True


We can reasonably conclude that we parsed all the events correctly because all of the overlapping fields from our correct dataset and our read-in dataset agree.

Let's save the final and intermediate datsets for future testing purposes

In [8]:
read_list_mode_data_to_bit_fields(bin_file).to_hdf('correct_bit_fields.hdf', 'df')
pixie16.read.read_list_mode_data([bin_file]).to_hdf('correct_events.hdf', 'df')

/Users/bmac3/Code/ATAP/atap-roots-controls/venv/lib/python3.8/site-packages/pandas/core/generic.py:2606: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['trace'], dtype='object')]

  pytables.to_hdf(
